# Wildfire and Drought Data Wrangling

A lot of the motivation behind my data wrangling came directly from the [weather, soil and drought Kaggle project](https://www.kaggle.com/datasets/cdminix/us-drought-meteorological-data). Christoph Minixhofer, the author of the Kaggle project, posted several [Jupyter notebooks on GitHub](https://github.com/MiniXC/droughted_scripts) on how he was able to gather the weather, soil and drought data from various United State goverment public APIs [<sup>2, 3, 4</sup>](#acknowledgements). I ended up reproducing a lot of his work because I wanted more fine grain weather, drought and soil data. His data coverered all of the United States and was only as narrow as each state's counties. I noticed a lot of the California counties were drastically different from each other in terms of terrain and weather patterns. I was worried that assuming each county was uniform was going to cause problems with the models.

I also downloaded the [latest curated fire data from the United States Forest Service](https://www.fs.usda.gov/rds/archive/Catalog/RDS-2013-0009.5) by Karen Short as of 2021 [<sup>1</sup>](#acknowledgements). I wanted more data to overlap with the weather data since the weather API only started at 2000 and the original [wildfire Kaggle project](https://www.kaggle.com/datasets/cdminix/us-drought-meteorological-data) only went to 2015. The newest Forest Service data went out 2018 and contained corrections to the orginal data.

### Data Wrangling Steps

  - &#x2611; Download [wildfire Sqlite DB](https://www.fs.usda.gov/rds/archive/Catalog/RDS-2013-0009.5) [<sup>`</sup>](#acknowledgements)
  - &#x2611; Download [soil CSV from Kaggle](https://www.kaggle.com/cdminix/us-drought-meteorological-data) which was originally sourced form the Harmonized World Soil Database [<sup>4</sup>](#acknowledgements)
  - &#x2611; Import soil CSV into Sqlite
  - &#x2611; Remove non-California data to keep the dataset more focused
  - &#x2611; Remove wildfire and soil/weather data that does not overlap (pre 2000 wildfires)
  - &#x2611; Load county FIPS codes and geospatial latitude, longitude and polygon perimeter [<sup>5</sup>](#acknowledgements) into Sqlite
  - Add indexes/foreign keys to speed up Sqlite
    - &#x2611; year
    - &#x2611; FIPS
    - &#x2611; long/lat on fires and soil
  - &#x2611; Truncate latitude and longitude to 1/10th degree (~11.11 km)
  - &#x2611; Backfill fires data missing the county by cross referencing the long/lat with each county's geospatial perimeter
  - &#x2611; Query NASA's weather API by date and long/lat between 2000-01-01 and 2018-12-31 [<sup>2</sup>](#acknowledgements)
  - &#x2611; Query U.S. Drought Monitor API by date and FIPS county between 2000-01-01 and 2018-12-31 [<sup>3</sup>](#acknowledgements)
  - &#x2611; Incidate if there was a prior fire at the same geospatial location within the last 1, 2, 3, 4 and 5 years respectively

#### Future Ideas
  - Running total of precipitation for past year (really slow)
  - Narrow down latitude and longitude to 1/10th degree (~11 km)
  - Download soil data by long/lat, instead of the current by county, [<sup>4</sup>](#acknowledgements)

In [1]:
%load_ext autoreload
%autoreload 2

from datetime import datetime, timedelta
import math
import numpy as np
import pandas as pd
import re
import requests
import shapely.wkt
from shapely.geometry import Point, Polygon
from sklearn.preprocessing import StandardScaler
import sqlite3
import time
import urllib.parse

#### Download California Counties

- Scrape Wikipedia for the Unites States counties.
- Filter out non-California counties.
- Truncate longitude and latitude to 1/10th degree or 6 minutes (~11.11 km). This should make the analysis go faster and also better generalize the location of predicted fires.
- Join the Wikipedia county data with the geographic boundies data for each California county. The geographic boundaries data is in the form of `MULTIPOLYGON (((` tuples that can be interpreted by the shapely python package. Later I use the geographic ploygons to backfill missing wildfire county data.

In [3]:
df_county = pd.read_html('https://en.wikipedia.org/wiki/User:Michael_J/County_table')[0]
float_degrees = lambda x: float(x.replace('°','').replace('–','-'))
df_county['latitude'] = df_county['Latitude'].apply(float_degrees)
df_county['longitude'] = df_county['Longitude'].apply(float_degrees)
df_county['lat'] = (round(df_county['latitude'] * 10) / 10)
df_county['long'] = (round(df_county['longitude'] * 10) / 10)
df_county['name'] = df_county['County [2]']

df_county = df_county[df_county['State'] == 'CA']
df_county = df_county.loc[:, df_county.columns.intersection(['FIPS', 'name', 'latitude', 'longitude', 'lat', 'long'])]

# Downloaded from https://data.edd.ca.gov/api/views/bpwh-bcb3/rows.csv?accessType=DOWNLOAD
county_geo_df = pd.read_csv('./county_geospatial.csv')
county_geo_df = county_geo_df.loc[:, county_geo_df.columns.intersection(['name', 'geo_multipolygon'])]

df_county = pd.merge(df_county, county_geo_df, left_on='name', right_on='name')
df_county = df_county.set_index('FIPS')

print(df_county.head())

       latitude   longitude   lat   long       name  \
FIPS                                                  
6001  37.648081 -121.913304  37.6 -121.9    Alameda   
6003  38.617610 -119.798999  38.6 -119.8     Alpine   
6005  38.443550 -120.653856  38.4 -120.7     Amador   
6007  39.665959 -121.601919  39.7 -121.6      Butte   
6009  38.187844 -120.555115  38.2 -120.6  Calaveras   

                                       geo_multipolygon  
FIPS                                                     
6001  MULTIPOLYGON (((-122.3110971410252 37.86340197...  
6003  MULTIPOLYGON (((-119.93538249202298 38.8084818...  
6005  MULTIPOLYGON (((-120.25874105290194 38.5799975...  
6007  MULTIPOLYGON (((-121.6354363647807 40.00088422...  
6009  MULTIPOLYGON (((-120.2108859831663 38.50000349...  


Load the counties DataFrame into Sqlite to make joins and analysis using SQL easier.

In [4]:
conn = sqlite3.connect('/Users/eerichmo/Documents/fires.sqlite')

cur = conn.cursor()

cur.execute('DROP TABLE county')
cur.execute('''CREATE TABLE county (
	fips	  					INTEGER NOT NULL,
	name	  					TEXT NOT NULL,
	latitude 					REAL NOT NULL,
	longitude					REAL NOT NULL,
	lat	    					REAL NOT NULL,
	long	  					REAL NOT NULL,
	geo_multipolygon	TEXT NOT NULL,
	PRIMARY KEY(fips)
);''')

df_county.to_sql('county', conn, if_exists='append')

conn.commit()
conn.close()

Calculate the geographic boundaries for California as a reference.

In [3]:
ca_bounds = [-180, 90, 180, -90]

for i, county in df_county.iterrows():
  name = county['name']
  geo = shapely.wkt.loads(county['geo_multipolygon'])

  # East
  if (geo.bounds[0] > ca_bounds[0]):
    ca_bounds[0] = geo.bounds[0]

  # South
  if (geo.bounds[1] < ca_bounds[1]):
    ca_bounds[1] = geo.bounds[1]

  # West
  if (geo.bounds[2] < ca_bounds[2]):
    ca_bounds[2] = geo.bounds[2]

  # Norht
  if (geo.bounds[3] > ca_bounds[3]):
    ca_bounds[3] = geo.bounds[3]

ca_bounds = tuple(ca_bounds)
print(f'California bounds (east-south, west-north): {ca_bounds}')

California bounds (east-south, west-north): (-116.10618166434291, 32.53402817678555, -123.51814169611895, 42.009834867689875)


### Adjust Forest Service Data

To make analysis easier I fiddled with the Forest Service Sqlite table to make the schema more consistent with other data sources. This included:

* Converting the discovery date `string` into the Sqlite ISO standard date `yyyy-mm-dd`
* Renaming several of the columns to be shorter such as `FIRE_YEAR` to `year`, `FIPS_CODE` to `fips`, `NWCG_GENERAL_CAUSE` to `cause`, etc
* Truncating `longitude` and `latitude` into 1/4th degree `long` and `lat` columns
* Lowercasing the column names
* Adding indexes on data and geospatial fields to help speed up the queries
* Renaming the fire causes to be shorter
* Adding a column for each fire's perimeter geospatial shape `geo_polygon` (only larger fires)
* Adding 5 new columns to hold prior fire data

In [5]:
conn = sqlite3.connect('/Users/eerichmo/Documents/fires.sqlite')
cur = conn.cursor()

cur.execute('ALTER TABLE fires ADD COLUMN year INTEGER NOT NULL DEFAULT 0')
cur.execute('UPDATE fires SET year = FIRE_YEAR WHERE year = 0')
cur.execute('ALTER TABLE fires DROP COLUMN FIRE_YEAR')

cur.execute('ALTER TABLE fires ADD COLUMN month INTEGER NOT NULL DEFAULT 0')
cur.execute("UPDATE fires set month = cast(substr(discovery_date, 1, instr(discovery_date,'/')-1) as 'INTEGER')")
cur.execute("ALTER TABLE fires ADD COLUMN day INTEGER NOT NULL DEFAULT 0")
cur.execute("UPDATE fires set day = cast(substr(substr(discovery_date, instr(discovery_date,'/')+1), 1, instr(substr(discovery_date, instr(discovery_date,'/')+1),'/')-1) as 'INTEGER')")
cur.execute("ALTER TABLE fires ADD COLUMN date TEXT NOT NULL DEFAULT ''")
cur.execute("UPDATE fires SET date = date(year|| '-' || substr('0' || month, -2, 2) || '-' || substr('0' || day, -2, 2))")

cur.execute('ALTER TABLE fires ADD COLUMN fips INTEGER NOT NULL DEFAULT 0')
cur.execute('UPDATE fires SET fips = FIPS_CODE WHERE FIPS = 0 and FIPS_CODE is not null')

cur.execute('ALTER TABLE fires ADD COLUMN long REAL NOT NULL DEFAULT 0')
cur.execute('ALTER TABLE fires ADD COLUMN lat REAL NOT NULL DEFAULT 0')
cur.execute('UPDATE fires SET long = round(LONGITUDE * 10) / 10, lat = round(LATITUDE * 10) / 10')

cur.execute("ALTER TABLE fires ADD COLUMN date_1d_before TEXT NOT NULL DEFAULT ''")
cur.execute("UPDATE fires SET date_1d_before = date(date, '-1 days') where date_1d_before = ''")
cur.execute("ALTER TABLE fires ADD COLUMN date_2d_before TEXT NOT NULL DEFAULT ''")
cur.execute("UPDATE fires SET date_2d_before = date(date, '-2 days') where date_2d_before = ''")
cur.execute("ALTER TABLE fires ADD COLUMN date_3d_before TEXT NOT NULL DEFAULT ''")
cur.execute("UPDATE fires SET date_3d_before = date(date, '-3 days') where date_3d_before = ''")

cur.execute('ALTER TABLE fires ADD COLUMN cause TEXT')
cur.execute("UPDATE fires SET cause = NWCG_GENERAL_CAUSE where cause is null")

cur.execute("UPDATE fires SET cause = 'Power' where cause = 'Power generation/transmission/distribution'")
cur.execute("UPDATE fires SET cause = 'Missing/Undefined' where cause = 'Missing data/not specified/undetermined'")
cur.execute("UPDATE fires SET cause = 'Equipment Use' where cause = 'Equipment and vehicle use'")
cur.execute("UPDATE fires SET cause = 'Arson' where cause = 'Arson/incendiarism'")
cur.execute("UPDATE fires SET cause = 'Children' where cause = 'Misuse of fire by a minor'")
cur.execute("UPDATE fires SET cause = 'Firearms' where cause = 'Firearms and explosives use'")
cur.execute("UPDATE fires SET cause = 'Railroad' where cause = 'Railroad operations and maintenance'")
cur.execute("UPDATE fires SET cause = 'Debris Burning' where cause = 'Debris and open burning'")
cur.execute("UPDATE fires SET cause = 'Recreation' where cause = 'Recreation and ceremony'")

cur.execute("ALTER TABLE fires ADD COLUMN geo_polygon TEXT")

cur.execute("ALTER TABLE fires ADD COLUMN prior_fire_0_1_year INTEGER NOT NULL DEFAULT 0")
cur.execute("ALTER TABLE fires ADD COLUMN prior_fire_1_2_year INTEGER NOT NULL DEFAULT 0")
cur.execute("ALTER TABLE fires ADD COLUMN prior_fire_2_3_year INTEGER NOT NULL DEFAULT 0")
cur.execute("ALTER TABLE fires ADD COLUMN prior_fire_3_4_year INTEGER NOT NULL DEFAULT 0")
cur.execute("ALTER TABLE fires ADD COLUMN prior_fire_4_5_year INTEGER NOT NULL DEFAULT 0")

cur.execute('DROP INDEX IF EXISTS idx_fires_fpa_id')
cur.execute('CREATE INDEX idx_fires_fpa_id ON fires(fpa_id)')

cur.execute('DROP INDEX IF EXISTS idx_fires_longitude_latitude')
cur.execute('CREATE INDEX idx_fires_longitude_latitude ON fires(longitude, latitude)')

cur.execute('DROP INDEX IF EXISTS idx_fires_date_long_lat')
cur.execute('CREATE INDEX idx_fires_date_long_lat ON fires(date, long, lat)')

cur.execute('DROP INDEX IF EXISTS idx_fires_date_range_long_lat')
cur.execute('CREATE INDEX idx_fires_date_range_long_lat ON fires(date, date_1d_before, date_2d_before, date_3d_before, long, lat)')

cur.execute('DROP INDEX IF EXISTS idx_fires_date_fips')
cur.execute('CREATE INDEX idx_fires_date_fips ON fires(date, fips)')

cur.execute('DROP INDEX IF EXISTS idx_fires_date_1d_before_fips')
cur.execute('CREATE INDEX idx_fires_date_1d_before_fips ON fires(date_1d_before, fips)')

cur.execute('DROP INDEX IF EXISTS idx_fires_date_2d_before_fips')
cur.execute('CREATE INDEX idx_fires_date_2d_before_fips ON fires(date_2d_before, fips)')

cur.execute('DROP INDEX IF EXISTS idx_fires_date_3d_before_fips')
cur.execute('CREATE INDEX idx_fires_date_3d_before_fips ON fires(date_3d_before, fips)')

cur.execute('DROP INDEX IF EXISTS idx_fires_date_1d_before_long_lat')
cur.execute('CREATE INDEX idx_fires_date_1d_before_long_lat ON fires(date_1d_before, long, lat)')

cur.execute('DROP INDEX IF EXISTS idx_fires_date_3d_before_long_lat')
cur.execute('CREATE INDEX idx_fires_date_2d_before_long_lat ON fires(date_2d_before, long, lat)')

cur.execute('DROP INDEX IF EXISTS idx_fires_date_3d_before_long_lat')
cur.execute('CREATE INDEX idx_fires_date_3d_before_long_lat ON fires(date_3d_before, long, lat)')

conn.commit()
conn.close()

Create a `weater_geo` table that holds the daily weather details at 1/4th degree or 27.77km wide longitude/latitude points between 1 Jan 2000 and 21 Dec 2018.

In [3]:
conn = sqlite3.connect('/Users/eerichmo/Documents/fires.sqlite')
cur = conn.cursor()

cur.execute('''CREATE TABLE weather_geo (
	date							      TEXT NOT NULL,
	year							      INTEGER NOT NULL,
	month							      INTEGER NOT NULL,
	day 							      INTEGER NOT NULL,
	long						        REAL NOT NULL,
	lat							        REAL NOT NULL,
	fips							      INTEGER NOT NULL,
	drought_score			      INTEGER NOT NULL DEFAULT 0,
	precipitation			      REAL NOT NULL,
	pressure					      REAL NOT NULL,
	humidity_2m				      REAL NOT NULL,
	temp_2m						      REAL NOT NULL,
	temp_dew_point_2m	      REAL NOT NULL,
	temp_wet_bulb_2m	      REAL NOT NULL,
	temp_max_2m				      REAL NOT NULL,
	temp_min_2m				      REAL NOT NULL,
	temp_range_2m			      REAL NOT NULL,
	temp_0m						      REAL NOT NULL,
	wind_10m					      REAL NOT NULL,
	wind_max_10m			      REAL NOT NULL,
	wind_min_10m			      REAL NOT NULL,
	wind_range_10m		      REAL NOT NULL,
	wind_50m					      REAL NOT NULL,
	wind_max_50m			      REAL NOT NULL,
	wind_min_50m			      REAL NOT NULL,
	wind_range_50m		      REAL NOT NULL,
	PRIMARY KEY(date, long, lat)
);''')

cur.execute('DROP INDEX IF EXISTS idx_weather_geo_fips')
cur.execute('CREATE INDEX idx_weather_geo_fips ON weather_geo (fips)')

cur.execute('DROP INDEX IF EXISTS idx_weather_geo_long_lat')
cur.execute('CREATE INDEX idx_weather_geo_long_lat ON weather_geo (long, lat)')

cur.execute('DROP INDEX IF EXISTS idx_weather_geo_month_long_lat')
cur.execute('CREATE INDEX idx_weather_geo_month_long_lat ON weather_geo (month, long, lat)')

cur.execute('DROP INDEX IF EXISTS idx_weather_geo_day')
cur.execute('CREATE INDEX idx_weather_geo_day ON weather_geo(day)')

conn.commit()
conn.close()

Defined the `fetch_weather` method for pulling various weather data points from NASA's POWER temperature and weather data API [<sup>2</sup>](#Acknowledgements).

In [14]:
weather_params = [p.strip() for p in re.findall(
"^\w+",
"""
WS10M_MIN      MERRA2 1/2x1/2 Minimum Wind Speed at 10 Meters (m/s) 
QV2M           MERRA2 1/2x1/2 Specific Humidity at 2 Meters (g/kg) 
T2M_RANGE      MERRA2 1/2x1/2 Temperature Range at 2 Meters (C) 
WS10M          MERRA2 1/2x1/2 Wind Speed at 10 Meters (m/s) 
T2M            MERRA2 1/2x1/2 Temperature at 2 Meters (C) 
WS50M_MIN      MERRA2 1/2x1/2 Minimum Wind Speed at 50 Meters (m/s) 
T2M_MAX        MERRA2 1/2x1/2 Maximum Temperature at 2 Meters (C) 
WS50M          MERRA2 1/2x1/2 Wind Speed at 50 Meters (m/s) 
TS             MERRA2 1/2x1/2 Earth Skin Temperature (C) 
WS50M_RANGE    MERRA2 1/2x1/2 Wind Speed Range at 50 Meters (m/s) 
WS50M_MAX      MERRA2 1/2x1/2 Maximum Wind Speed at 50 Meters (m/s) 
WS10M_MAX      MERRA2 1/2x1/2 Maximum Wind Speed at 10 Meters (m/s) 
WS10M_RANGE    MERRA2 1/2x1/2 Wind Speed Range at 10 Meters (m/s) 
PS             MERRA2 1/2x1/2 Surface Pressure (kPa) 
T2MDEW         MERRA2 1/2x1/2 Dew/Frost Point at 2 Meters (C) 
T2M_MIN        MERRA2 1/2x1/2 Minimum Temperature at 2 Meters (C) 
T2MWET         MERRA2 1/2x1/2 Wet Bulb Temperature at 2 Meters (C) 
PRECTOT        MERRA2 1/2x1/2 Precipitation (mm day-1) 
""",
re.MULTILINE
)]

print(weather_params)

def fetch_weather(long, lat, start, end):
    return requests.get(
      'https://power.larc.nasa.gov/api/temporal/daily/point',
      {
          'parameters': ','.join(weather_params),
          'community': 'SB',
          'longitude': long,
          'latitude': lat,
          'start': start,
          'end': end,
          'format': 'JSON',
      }
    ).json()['properties']['parameter']

['WS10M_MIN', 'QV2M', 'T2M_RANGE', 'WS10M', 'T2M', 'WS50M_MIN', 'T2M_MAX', 'WS50M', 'TS', 'WS50M_RANGE', 'WS50M_MAX', 'WS10M_MAX', 'WS10M_RANGE', 'PS', 'T2MDEW', 'T2M_MIN', 'T2MWET', 'PRECTOT']


For each California county iterate over all the 11.11km (1/10th degree or 6 minutes) longitude and latitude points within that county's goegraphical boundary and fetch the weather data for those points between 1 Jan 2000 and 31 Dec 2018.

In [21]:
start_date = '20000101'
end_date = '20181231'

conn = sqlite3.connect('/Users/eerichmo/Documents/fires.sqlite')
cur = conn.cursor()

for fips, county in df_county.iterrows():
  name = county.name
  geo = shapely.wkt.loads(county.geo_multipolygon)

  long_min = round(geo.bounds[0], 1)
  long_max = round(geo.bounds[2], 1)

  lat_min = round(geo.bounds[1], 1)
  lat_max = round(geo.bounds[3], 1)

  print(f'{name} southwest to northeast: ({long_min}, {lat_min}) to ({long_max}, {lat_max})')

  for long in range(int(long_min * 10), int(long_max * 10) + 1):
    for lat in range(int(lat_min * 10), int(lat_max * 10) + 1):
      if ((long % 10 == 0 or long % 10 == 5) and (lat % 10 == 0 or lat % 10 == 5)):
        continue

      point = Point(long / 10, lat / 10)
      start = time.time()

      if geo.contains(point):
        # Only process lat/long that have not been precessed yet.
        cur.execute("""SELECT 1 as found FROM weather_geo WHERE lat = :lat and long = :long""", { 'lat': point.y, 'long': point.x })
        found = cur.fetchall()

        if (len(found) == 0):
          print(f'lat: {point.x}, long: {point.y}')
          json = fetch_weather(point.x, point.y, start_date, end_date)

          for date in json['TS'].keys():
            cur.execute('''
              INSERT INTO weather_geo (
                date, year, month, day, long, lat, fips, precipitation, pressure, humidity_2m,
                temp_2m, temp_dew_point_2m, temp_wet_bulb_2m, temp_max_2m, temp_min_2m, temp_range_2m,
                temp_0m, wind_10m, wind_max_10m, wind_min_10m, wind_range_10m, wind_50m,
                wind_max_50m, wind_min_50m, wind_range_50m
              )
              VALUES (
                :date, :year, :month, :day, :long, :lat, :fips, :precipitation, :pressure, :humidity_2m,
                :temp_2m, :temp_dew_point_2m, :temp_wet_bulb_2m, :temp_max_2m, :temp_min_2m, :temp_range_2m,
                :temp_0m, :wind_10m, :wind_max_10m, :wind_min_10m, :wind_range_10m, :wind_50m,
                :wind_max_50m, :wind_min_50m, :wind_range_50m
              )
              ''', {
                'date': f'{date[0:4]}-{date[4:6]}-{date[6:8]}',
                'year': int(date[0:4]),
                'month': int(date[4:6]),
                'day': int(date[6:8]),
                'long': point.x,
                'lat': point.y,
                'fips': fips,
                'precipitation': json['PRECTOTCORR'][date],
                'pressure': json['PS'][date],
                'humidity_2m': json['QV2M'][date],
                'temp_2m': json['T2M'][date],
                'temp_dew_point_2m': json['T2MDEW'][date],
                'temp_wet_bulb_2m': json['T2MWET'][date],
                'temp_max_2m': json['T2M_MAX'][date],
                'temp_min_2m': json['T2M_MIN'][date],
                'temp_range_2m': json['T2M_RANGE'][date],
                'temp_0m': json['TS'][date],
                'wind_10m': json['WS10M'][date],
                'wind_max_10m': json['WS10M_MAX'][date],
                'wind_min_10m': json['WS10M_MIN'][date],
                'wind_range_10m': json['WS10M_RANGE'][date],
                'wind_50m': json['WS50M'][date],
                'wind_max_50m': json['WS50M_MAX'][date],
                'wind_min_50m': json['WS50M_MIN'][date],
                'wind_range_50m': json['WS50M_RANGE'][date]
              })

            conn.commit()

          end = time.time()
          print(f'{name} at {point} took {round(end - start, 1)}s')

conn.close()

6001 southwest to northeast: (-122.3, 37.5) to (-121.5, 37.9)
6003 southwest to northeast: (-120.1, 38.3) to (-119.5, 38.9)
6005 southwest to northeast: (-121.0, 38.2) to (-120.1, 38.7)
lat: -120.8, long: 38.4
6005 at POINT (-120.8 38.4) took 10.4s
lat: -120.8, long: 38.5
6005 at POINT (-120.8 38.5) took 10.2s
lat: -120.7, long: 38.4
6005 at POINT (-120.7 38.4) took 9.6s
lat: -120.7, long: 38.5
6005 at POINT (-120.7 38.5) took 10.0s
lat: -120.6, long: 38.4
6005 at POINT (-120.6 38.4) took 10.5s
lat: -120.6, long: 38.5
6005 at POINT (-120.6 38.5) took 10.7s
lat: -120.4, long: 38.5
6005 at POINT (-120.4 38.5) took 9.5s
lat: -120.3, long: 38.5
6005 at POINT (-120.3 38.5) took 10.2s
lat: -120.2, long: 38.5
6005 at POINT (-120.2 38.5) took 9.7s
lat: -120.2, long: 38.6
6005 at POINT (-120.2 38.6) took 69.7s
lat: -120.1, long: 38.6
6005 at POINT (-120.1 38.6) took 10.4s
lat: -120.1, long: 38.7
6005 at POINT (-120.1 38.7) took 10.0s
6007 southwest to northeast: (-122.1, 39.3) to (-121.1, 40.2)

Load the `soil.csv`, from the [drought and soil Kaggle project](https://www.kaggle.com/datasets/cdminix/us-drought-meteorological-data?select=soil_data.csv), into Sqlite. The original source of the data is the Harmonized World Soil Database v 1.2[<sup>4</sup>](#Acknowledgements).

In [328]:
conn = sqlite3.connect('/Users/eerichmo/Documents/fires.sqlite')
cur = conn.cursor()

cur.execute('DROP TABLE IF EXISTS soil')
cur.execute('''CREATE TABLE soil (
	long										REAL NOT NULL,
	lat											REAL NOT NULL,
	fips										INTEGER NOT NULL,
	latitude								REAL NOT NULL,
	longitude								REAL NOT NULL,
	elevation								INTEGER NOT NULL,
	slope_005								REAL NOT NULL,
	slope_005_02						REAL NOT NULL,
	slope_02_05							REAL NOT NULL,
	slope_05_10							REAL NOT NULL,
	slope_10_15							REAL NOT NULL,
	slope_15_30							REAL NOT NULL,
	slope_30_45							REAL NOT NULL,
	slope_45								REAL NOT NULL,
	aspect_north						REAL NOT NULL,
	aspect_east							REAL NOT NULL,
	aspect_south						REAL NOT NULL,
	aspect_west							REAL NOT NULL,
	aspect_unknown					REAL NOT NULL,
	water_land							REAL NOT NULL,
	barren_land							REAL NOT NULL,
	urban_land							REAL NOT NULL,
	grass_land							REAL NOT NULL,
	forest_land							REAL NOT NULL,
	partial_cultivated_land	REAL NOT NULL,
	irrigated_land					REAL NOT NULL,
	cultivated_land					REAL NOT NULL,
	nutrient								INTEGER NOT NULL,
	nutrient_retention			INTEGER NOT NULL,
	rooting									INTEGER NOT NULL,
	oxygen									INTEGER NOT NULL,
	excess_salts						INTEGER NOT NULL,
	toxicity								INTEGER NOT NULL,
	workablity							INTEGER NOT NULL
)''')

cur.execute('DROP INDEX IF EXISTS idx_soil_fips')
cur.execute('CREATE INDEX idx_soil_fips ON soil(fips)')

cur.execute('DROP INDEX IF EXISTS idx_soil_lat_long')
cur.execute('CREATE INDEX idx_soil_lat_long ON soil(lat, long)')

soil_df = pd.read_csv('./soil.csv')
soil_df['lat'] = round(soil_df['latitude'] * 10) / 10
soil_df['long'] = round(soil_df['longitude'] * 10) / 10

soil_df = soil_df[soil_df['fips'].isin(df_county.index)]

soil_df.to_sql('soil', conn, if_exists='append', index=False)

conn.commit()
conn.close()

Create a `drought` table for holding the drought score for all California counties between 1 Jan 2000 and 21 Dec 2018.

In [11]:
conn = sqlite3.connect('/Users/eerichmo/Documents/fires.sqlite')
cur = conn.cursor()

cur.execute('DROP TABLE IF EXISTS drought')
cur.execute('''CREATE TABLE drought (
  date          TEXT NOT NULL,
  fips          INTEGER NOT NULL,
  drought_score REAL,
  PRIMARY KEY(date, fips)
)''')

conn.commit()
conn.close()

Pull the drought scores from the [US Drought Monitor website](https://droughtmonitor.unl.edu/).

In [28]:
import requests

def fetch_drought(fips):
    return requests.get(
        'https://usdmdataservices.unl.edu/api/CountyStatistics/GetDroughtSeverityStatisticsByAreaPercent',
        {
            'aoi': fips,
            'startdate': '10/1/1999',
            'enddate': '12/31/2018',
            'statisticsType': 1,
        }
    ).json()

For each county that doesn't have a drought score pull the drought score from US Drought Monintor.

In [29]:
conn = sqlite3.connect('/Users/eerichmo/Documents/fires.sqlite')
cur = conn.cursor()

cur.execute('SELECT DISTINCT fips FROM drought WHERE drought_score IS NULL')

for row in cur.fetchall():
  fips = row[0]
  fips_5_char = f'0{str(fips)}' if fips < 10000 else str(fips)

  print(f'Fetch drought score for {fips_5_char}')
  json = fetch_drought(fips_5_char)

  for item in json:
    drought_score = float(item['D0'])/100 + float(item['D1'])/100 + float(item['D2'])/100 + float(item['D3'])/100 + float(item['D4'])/100

    # Backfill Jan 4 score to Jan 1-3 of 2000 as it seems to be missing
    start = '2000-01-01' if item['ValidStart'] <= '2000-01-04' else item['ValidStart']

    drought_params = { 'fips': fips, 'drought_score': drought_score, 'start': start, 'end': item['ValidEnd'] }
    
    cur.execute('''
      UPDATE drought SET
        drought_score = :drought_score
      WHERE
        fips = :fips AND date >= :start AND date <= :end
    ''', drought_params)

  conn.commit()
  
conn.close()


Fetch drought score for 06001
Fetch drought score for 06003
Fetch drought score for 06005
Fetch drought score for 06007
Fetch drought score for 06009
Fetch drought score for 06011
Fetch drought score for 06013
Fetch drought score for 06015
Fetch drought score for 06017
Fetch drought score for 06019
Fetch drought score for 06021
Fetch drought score for 06023
Fetch drought score for 06025
Fetch drought score for 06027
Fetch drought score for 06029
Fetch drought score for 06031
Fetch drought score for 06033
Fetch drought score for 06035
Fetch drought score for 06037
Fetch drought score for 06039
Fetch drought score for 06041
Fetch drought score for 06043
Fetch drought score for 06045
Fetch drought score for 06047
Fetch drought score for 06049
Fetch drought score for 06051
Fetch drought score for 06053
Fetch drought score for 06055
Fetch drought score for 06057
Fetch drought score for 06059
Fetch drought score for 06061
Fetch drought score for 06063
Fetch drought score for 06065
Fetch drou

Backfill any missing county identifiers (FIPS codes) for California fires.

In [78]:
conn = sqlite3.connect('/Users/eerichmo/Documents/fires.sqlite')
cur = conn.cursor()

cur.execute('SELECT longitude, latitude FROM fires WHERE fips = 0 order by longitude, latitude')

for row in cur.fetchall():
  long = row[0]
  lat = row[1]
  found = False
  min_dist = 180
  closest_fips = 0

  for fips, county in df_county.iterrows():
    region = shapely.wkt.loads(county['geo_multipolygon'])
    point = Point(long, lat)

    if region.contains(point):
      print(f'{point} is in {fips}')
      cur.execute('''
        UPDATE fires SET fips = :fips
        WHERE longitude = :longitude AND latitude = :latitude
      ''', { 'fips': fips, 'longitude': long, 'latitude': lat })
      conn.commit()
      found = True
      break

    dist = region.boundary.distance(point)

    if min_dist > dist:
      min_dist = dist
      closest_fips = fips

  if not found:
    print(f'{point} not found. Closest county, by {round(min_dist, 3)}, is {closest_fips}')
    cur.execute('''
      UPDATE fires SET fips = :fips
      WHERE longitude = :longitude AND latitude = :latitude
    ''', { 'fips': closest_fips, 'longitude': long, 'latitude': lat })
    conn.commit()
  
conn.close()

In [ ]:
conn = sqlite3.connect('/Users/eerichmo/Documents/fires.sqlite')
cur = conn.cursor()

df = pd.read_sql_query('select * from fires', conn)

final = df.isna().sum()
cols = []
for count, col in zip(final,list(df.columns)):
    if count > 0:
        cols.append(col)

print(f'Columns with null: {cols}')

conn.close()

In [333]:
conn = sqlite3.connect('/Users/eerichmo/Documents/fires.sqlite')
cur = conn.cursor()

cur.execute("""UPDATE weather_county SET drought_score = (
  select drought_score
  from drought
  where
    drought.date = weather_county.date
    and drought.fips = weather_county.fips
)
where
  weather_county.drought_score is null
""")

cur.execute("""UPDATE weather_geo SET drought_score = (
  select drought_score
  from drought
  where
    drought.date = weather_geo.date
    and drought.fips = weather_geo.fips
)
where
  weather_geo.drought_score = 0
""")

conn.commit()
conn.close()

In [227]:
conn = sqlite3.connect('/Users/eerichmo/Documents/fires.sqlite')
cur = conn.cursor()

cur.execute('ALTER TABLE weather_county ADD COLUMN month INTEGER NOT NULL DEFAULT 0')
cur.execute("""UPDATE weather_county SET month = CAST(strftime('%m', date) as 'INTEGER') WHERE month = 0""")

cur.execute('DROP INDEX IF EXISTS idx_weather_county_fips_date')
cur.execute('CREATE INDEX idx_weather_county_fips_date ON weather_county(date, fips)')

conn.commit()
conn.close()

Load the geospatial perimeter from the California State GeoPortal API[<sup>6</sup>](#acknowledgements) For large fires (class `D` of 100+ acres)

In [89]:
import urllib.parse
import re
from datetime import datetime, timedelta
from shapely.geometry import Polygon
from shapely import wkt

conn = sqlite3.connect('/Users/eerichmo/Documents/fires.sqlite')
cur = conn.cursor()

df_fires = pd.read_sql_query("""
select
  fpa_id, longitude, latitude, year, date, fire_name, fire_size,
  ics_209_plus_incident_join_id, local_incident_id, local_fire_report_id, nwcg_reporting_unit_id
from fires
where
  geo_polygon is null
  and fire_size_class >= 'D'
  and (
    fire_name is not null
    or local_incident_id is not null
    or ics_209_plus_incident_join_id like '%_%'
  )
order by date
""", conn)

url = 'https://egis.fire.ca.gov/arcgis/rest/services/FRAP/FirePerimeters_FS/FeatureServer/0/query?outFields=*&outSR=4326&f=json'

for i, fire in df_fires.iterrows():
  fire_name = (fire.fire_name or 'N/A').replace("'", "''").replace(" COMPLEX", "").replace(" LIGHTNING", "")
  fire_name_alt = 'N/A'
  report_id = str(fire.local_fire_report_id or 'N/A')

  regex_match = re.search(r'_(CA|NV)-([^-]+)-([^-]+)_([^_]+)', fire.ics_209_plus_incident_join_id or '')
  if (regex_match):
    unit_id = regex_match.group(2)
    incident_id = regex_match.group(3) or 'N/A'
    fire_name_alt = (regex_match.group(4) or 'N/A').replace("'", "''").replace(" COMPLEX", "").replace(" LIGHTNING", "")
  else:
    regex_match = re.search(r'^([A-Z]{3})(\d+)', fire.local_incident_id or '')
    if (regex_match):
      unit_id = regex_match.group(1)
      incident_id = regex_match.group(2) or 'N/A'
    else:
      unit_id = (fire.nwcg_reporting_unit_id or '').replace('USCA', '')
      incident_id = str(fire.ics_209_plus_incident_join_id or fire.local_incident_id or fire.local_fire_report_id or 'N/A').replace(unit_id, '')

  date = datetime.strptime(fire.date, '%Y-%m-%d')
  date_before = date + timedelta(days=-1)
  date_after = date + timedelta(days=1)
  date_range = f"ALARM_DATE>=DATE '{datetime.strftime(date_before, '%Y-%m-%d')}' and ALARM_DATE<=DATE '{datetime.strftime(date_after, '%Y-%m-%d')}'"

  geo_query = f"geometryType=esriGeometryPoint&geometry={fire.longitude},{fire.latitude}&spatialRel=esriSpatialRelIntersects&inSR=4326"

  where = urllib.parse.quote(f"{date_range} \
and STATE='CA' \
and ( \
  UNIT_ID='{unit_id}' or REPORT_AC={fire.fire_size or -1} or FIRE_NAME like '{fire_name}%' or FIRE_NAME like '{fire_name_alt}%' \
  or INC_NUM='{incident_id.zfill(8)}' or INC_NUM='{incident_id.zfill(6)}' or INC_NUM='{report_id.zfill(8)}' or FIRE_NUM='{report_id.zfill(6)}' )")
  result = requests.get(f'{url}&where={where}&{geo_query}').json()['features']

  if (len(result) == 0):
    where = urllib.parse.quote(f"{date_range} \
  and STATE='CA' \
  and ( UNIT_ID='{unit_id}' or REPORT_AC={fire.fire_size or -1} ) \
  and ( FIRE_NAME='{fire_name}' or FIRE_NAME='{fire_name_alt}' or INC_NUM='{incident_id.zfill(8)}' or INC_NUM='{incident_id.zfill(6)}' or INC_NUM='{report_id.zfill(8)}' or FIRE_NUM='{report_id.zfill(6)}' )")
    result = requests.get(f'{url}&where={where}').json()['features']

  if (len(result) == 0):
    where = urllib.parse.quote(f"{date_range} \
and STATE='CA' \
and ( INC_NUM='{incident_id.zfill(8)}' or INC_NUM='{incident_id.zfill(6)}' or INC_NUM='{report_id.zfill(8)}' or FIRE_NUM='{report_id.zfill(6)}' or REPORT_AC={fire.fire_size or -1} ) \
and ( FIRE_NAME like '{fire_name}%' or FIRE_NAME like '{fire_name_alt}%' )")
    result = requests.get(f'{url}&where={where}').json()['features']

  if (len(result) == 0):
    where = urllib.parse.quote(f"YEAR_={fire.year} \
and STATE='CA' \
and UNIT_ID='{unit_id}' \
and ( FIRE_NAME='{fire_name}' or FIRE_NAME='{fire_name_alt}' ) \
and ( REPORT_AC={fire.fire_size or -1} or INC_NUM='{incident_id.zfill(8)}' or INC_NUM='{incident_id.zfill(6)}' or INC_NUM='{report_id.zfill(8)}' or FIRE_NUM='{report_id.zfill(6)}' )")
    result = requests.get(f'{url}&where={where}').json()['features']

  if (len(result) >= 1):
    geo_json = result[0]['geometry']['rings'][0]
    geo = Polygon(geo_json)

    print(f'Saved {len(result)} match(es) for {fire.fpa_id if fire_name == "N/A" else fire_name} --- {where} {geo_query}')
    cur.execute("""
    update fires set geo_polygon = :polygon where fpa_id = :fpa_id
    """, { 'polygon': wkt.dumps(geo), 'fpa_id': fire.fpa_id })

    conn.commit()
  # else:
    # print(f'MISSING: {fire_name} --- {geo_query}')
    # print(f'Could not find {fire.fire_name} on {date_before} - {date_after} ({incident_id}, size: {fire.fire_size})')

conn.close()

Saved HILL --- ALARM_DATE%3E%3DDATE%20%272012-06-22%27%20and%20ALARM_DATE%3C%3DDATE%20%272012-06-24%27%20%20%20and%20STATE%3D%27CA%27%20%20%20and%20%28%20UNIT_ID%3D%27LPF%27%20or%20REPORT_AC%3D689.0%20%29%20%20%20and%20%28%20FIRE_NAME%3D%27HILL%27%20or%20FIRE_NAME%3D%27HILL%27%20or%20INC_NUM%3D%2700001505%27%20or%20INC_NUM%3D%27001505%27%20or%20INC_NUM%3D%27000021.0%27%20or%20FIRE_NUM%3D%270021.0%27%20%29 geometryType=esriGeometryPoint&geometry=-118.8683333,34.73111111&spatialRel=esriSpatialRelIntersects&inSR=4326


Backfill prior fires at the same longitude and latitude for 1, 2, 3, 4 and 5 years back respectively.

In [5]:
conn = sqlite3.connect('/Users/eerichmo/Documents/fires.sqlite')
cur = conn.cursor()

cur.execute("""UPDATE fires SET prior_fire_0_1_year = 1
where
  exists(
    select 1
    from fires prior
    where
      prior.longitude = fires.longitude
      and prior.latitude = fires.latitude
      and julianday(fires.date) - julianday(prior.date) > 30
	    and julianday(fires.date) - julianday(prior.date) <= 365
  )
""")

cur.execute("""UPDATE fires SET prior_fire_1_2_year = 1
where
  exists(
    select 1
    from fires prior
    where
      prior.longitude = fires.longitude
      and prior.latitude = fires.latitude
      and julianday(fires.date) - julianday(prior.date) > 365
	    and julianday(fires.date) - julianday(prior.date) <= (365 * 2)
  )
""")

cur.execute("""UPDATE fires SET prior_fire_2_3_year = 1
where
  exists(
    select 1
    from fires prior
    where
      prior.longitude = fires.longitude
      and prior.latitude = fires.latitude
      and julianday(fires.date) - julianday(prior.date) > (365 * 2)
	    and julianday(fires.date) - julianday(prior.date) <= (365 * 3)
  )
""")

cur.execute("""UPDATE fires SET prior_fire_3_4_year = 1
where
  exists(
    select 1
    from fires prior
    where
      prior.longitude = fires.longitude
      and prior.latitude = fires.latitude
      and julianday(fires.date) - julianday(prior.date) > (365 * 3)
	    and julianday(fires.date) - julianday(prior.date) <= (365 * 4)
  )
""")

cur.execute("""UPDATE fires SET prior_fire_4_5_year = 1
where
  exists(
    select 1
    from fires prior
    where
      prior.longitude = fires.longitude
      and prior.latitude = fires.latitude
      and julianday(fires.date) - julianday(prior.date) > (365 * 4)
	    and julianday(fires.date) - julianday(prior.date) <= (365 * 5)
  )
""")

conn.commit()
conn.close()

Backfill fires if the fire's longitude and latitude origin was within the perimeter of the prior fire for the last 1, 2, 3, 4 and 5 years respectively.

In [90]:
import time
import numpy as np

conn = sqlite3.connect('/Users/eerichmo/Documents/fires.sqlite')
cur = conn.cursor()

df_prior_all = pd.read_sql_query("""
  select date, geo_polygon from fires
  where geo_polygon is not null
  order by date
""", conn, parse_dates={'date': {'format': '%Y-%m-%d'}})

def to_geo(pair):
  return (pair.date, shapely.wkt.loads(pair.geo_polygon))

prior_all = tuple(map(to_geo, df_prior_all.itertuples()))

df_fires = pd.read_sql_query("""
  select fpa_id, fire_name, date, longitude, latitude from fires
  order by date
""", conn, index_col="fpa_id", parse_dates={'date': {'format': '%Y-%m-%d'}})

for fire_id, fire in df_fires.iterrows():
  start = time.time()
  found = False
  fire_pt = Point(fire.longitude, fire.latitude)

  for year_back in range(0, 5):
    max_date = fire.date - pd.DateOffset(days=(365 * year_back))
    min_date = fire.date - pd.DateOffset(days=(365 * (year_back + 1)))

    for prior in prior_all:
      prior_date = prior[0]
      
      if (prior_date < max_date and prior_date >= min_date):
        prior_perimeter = prior[1]

        if prior_perimeter.contains(fire_pt):
          print(f'{fire.fire_name or fire_id} on {fire.date.strftime("%Y-%m-%d")} was within prior fire perimeter {year_back}-{year_back+1} years before')
          cur.execute(f"UPDATE fires SET prior_fire_{year_back}_{year_back+1}_year = 1 WHERE fpa_id = :fpa_id", { 'fpa_id': fire_id })
          conn.commit()
          found = True
          break
      
      if (prior_date >= fire.date):
        break

  if (found):
    end = time.time()
    print(f'{fire.fire_name or fire_id} took {round(end - start, 3)}s')

conn.close()

DEL LOMA on 2010-08-25 was within prior fire perimeter 2-3 years before
DEL LOMA took 0.028s
GOBBLER on 2010-08-25 was within prior fire perimeter 0-1 years before
GOBBLER took 0.04s
WEBER on 2010-08-25 was within prior fire perimeter 2-3 years before
WEBER took 0.028s
MISSION on 2010-08-25 was within prior fire perimeter 2-3 years before
MISSION took 0.028s
MESA on 2010-08-25 was within prior fire perimeter 2-3 years before
MESA took 0.028s
CASTILLE on 2010-08-26 was within prior fire perimeter 3-4 years before
CASTILLE took 0.028s
CASTILLE CPLX POPPET on 2010-08-26 was within prior fire perimeter 3-4 years before
CASTILLE CPLX POPPET took 0.028s
DEVIL on 2010-09-02 was within prior fire perimeter 1-2 years before
DEVIL took 0.029s
KINGSFORD on 2010-09-06 was within prior fire perimeter 2-3 years before
KINGSFORD took 0.028s
SFO-2010-CACDFTCU020705 on 2010-09-13 was within prior fire perimeter 1-2 years before
SFO-2010-CACDFTCU020705 took 1.499s
HWY 243 on 2010-09-20 was within prior 

Add a rank for each fire cause so that natural/accidental causes are prioritized as the model only deals with fires that can be predicted based on natural weather and soil events.

In [4]:
conn = sqlite3.connect('/Users/eerichmo/Documents/fires.sqlite')
cur = conn.cursor()

cur.execute('DROP TABLE IF EXISTS cause_rank')
cur.execute("""
	CREATE TABLE cause_rank (
		rank		INTEGER NOT NULL DEFAULT 0,
		cause		TEXT NOT NULL,
		PRIMARY KEY (cause)
	)
""")

# Rank by natural or made more likely by natural conditions (ie drought)
cur.execute("""
INSERT INTO cause_rank (cause, rank)
select 'Natural', 1 union
select 'Power', 2 union
select 'Recreation', 3 union
select 'Other causes', 4 union
select 'Smoking', 5 union
select 'Railroad', 6 union
select 'Equipment Use', 7 union
select 'Missing/Undefined', 8 union
select 'Children', 9 union
select 'Debris Burning', 10 union
select 'Firearms', 11 union
select 'Fireworks', 12 union
select 'Arson', 13
""")

cur.execute('DROP TABLE fires_rollup')
cur.execute("""
	CREATE TABLE fires_rollup AS
		select
			date,
			long,
			lat,
			cause,
			fire_size_class,
			prior_fire_0_1_year,
			prior_fire_1_2_year,
			prior_fire_2_3_year,
			prior_fire_3_4_year,
			prior_fire_4_5_year
		from (
			select
				date,
				long,
				lat,
				min(rank) as cause_rank,
				max(fire_size_class) as fire_size_class,
				max(prior_fire_0_1_year) as prior_fire_0_1_year,
				max(prior_fire_1_2_year) as prior_fire_1_2_year,
				max(prior_fire_2_3_year) as prior_fire_2_3_year,
				max(prior_fire_3_4_year) as prior_fire_3_4_year,
				max(prior_fire_4_5_year) as prior_fire_4_5_year
			from (
				-- Join date and 1, 2 and 3 days earlier in case the discovery date is delayed
				select date, long, lat, cause, fire_size_class, prior_fire_0_1_year, prior_fire_1_2_year, prior_fire_2_3_year, prior_fire_3_4_year, prior_fire_4_5_year
				from fires
				union
				select date_1d_before as date, long, lat, cause, fire_size_class, prior_fire_0_1_year, prior_fire_1_2_year, prior_fire_2_3_year, prior_fire_3_4_year, prior_fire_4_5_year
				from fires
				union
				select date_2d_before as date, long, lat, cause, fire_size_class, prior_fire_0_1_year, prior_fire_1_2_year, prior_fire_2_3_year, prior_fire_3_4_year, prior_fire_4_5_year
				from fires
				union
				select date_3d_before as date, long, lat, cause, fire_size_class, prior_fire_0_1_year, prior_fire_1_2_year, prior_fire_2_3_year, prior_fire_3_4_year, prior_fire_4_5_year
				from fires
			) as fires
			inner join cause_rank
				on cause_rank.cause = fires.cause
			group by date, long, lat
		) as fires
		left join cause_rank
			on cause_rank.rank = fires.cause_rank
""")

cur.execute('DROP INDEX IF EXISTS idx_fires_rollup_date_long_lat')
cur.execute('CREATE INDEX idx_fires_rollup_date_long_lat ON fires_rollup(date, long, lat)')

conn.commit()
conn.close()

In [10]:
conn = sqlite3.connect('/Users/eerichmo/Documents/fires.sqlite')
cur = conn.cursor()

cur.execute('DROP TABLE IF EXISTS weather_geo_no_fire')
cur.execute("""
create table weather_geo_no_fire as
	select *, 0 as prior_fire_0_1_year, 0 as prior_fire_1_2_year, 0 as prior_fire_2_3_year, 0 as prior_fire_3_4_year, 0 as prior_fire_4_5_year
	from weather_geo
	where not exists(
		select 1
		from fires_rollup
		where
			fires_rollup.date = weather_geo.date
			and fires_rollup.long = weather_geo.long
			and fires_rollup.lat = weather_geo.lat
	)
order by random()
""")

cur.execute('DROP INDEX IF EXISTS idx_weather_geo_no_fire_fips')
cur.execute('CREATE INDEX idx_weather_geo_no_fire_fips ON weather_geo_no_fire(fips)')

cur.execute('DROP INDEX IF EXISTS idx_weather_geo_no_fire_date_long_lat')
cur.execute('CREATE INDEX idx_weather_geo_no_fire_date_long_lat ON weather_geo_no_fire(date, long, lat)')

cur.execute('DROP INDEX IF EXISTS idx_weather_geo_no_fire_long_lat')
cur.execute('CREATE INDEX idx_weather_geo_no_fire_long_lat ON weather_geo_no_fire(long, lat)')

conn.close()

In [13]:
conn = sqlite3.connect('/Users/eerichmo/Documents/fires.sqlite')
cur = conn.cursor()

one_year_in_sec = 365 * 24 * 60 * 60

def diff_years(date1, date2):
	return math.floor((date1 - date2) / one_year_in_sec)

for fire_year in range(2000, 2019):
	print(f'Fire year {fire_year}')
	df_fires_large = pd.read_sql_query("""
	select long, lat, strftime('%s', date) as date_in_sec
	from fires_rollup
	where
		fire_size_class >= 'D'
		and date <= :max_date
		and date >= :min_date
	order by long, lat, date
	""", conn, params = {'max_date': f'{fire_year}-12-31', 'min_date': f'{fire_year-5}-01-01'})

	df_fires_large['date_in_sec'] = df_fires_large.date_in_sec.astype(int)

	fire_dict = df_fires_large.groupby(['long', 'lat'])['date_in_sec'].apply(np.array).to_dict()

	df_weather = pd.read_sql_query("""
	select rowid, long, lat, strftime('%s', date) as date_in_sec
	from weather_geo_no_fire
	where
		year = :fire_year
	""", conn, index_col=['rowid'], params = {'fire_year': fire_year})

	i = 0

	for rowid, row in df_weather.iterrows():
		pt = (row.long, row.lat)
		fire_dates = fire_dict.get(pt)

		if fire_dates is not None:
			date_in_sec = int(row.date_in_sec)
			five_years_before_in_sec = date_in_sec - (one_year_in_sec * 5)
			matches = fire_dates[(fire_dates <= date_in_sec) & (fire_dates >= five_years_before_in_sec)]	
			years_back = np.unique([diff_years(date_in_sec, match) for match in matches])

			sql = ""
			for year in years_back:
				if year >=0 and year < 5:
					sql += f" prior_fire_{year}_{year+1}_year = 1,"

			if sql != "":
				if i % 10000 == 0:
					print(f"UPDATE weather_geo_no_fire SET {sql[:-1]} WHERE rowid = {rowid}")
				cur.execute(f"UPDATE weather_geo_no_fire SET {sql[:-1]} WHERE rowid = {rowid}")
				conn.commit()
				i += 1

conn.close()

Fire year 2003
UPDATE weather_geo_no_fire SET  prior_fire_0_1_year = 1 WHERE rowid = 3
UPDATE weather_geo_no_fire SET  prior_fire_2_3_year = 1 WHERE rowid = 1825186
UPDATE weather_geo_no_fire SET  prior_fire_0_1_year = 1 WHERE rowid = 3641313
UPDATE weather_geo_no_fire SET  prior_fire_0_1_year = 1, prior_fire_2_3_year = 1 WHERE rowid = 5455520
UPDATE weather_geo_no_fire SET  prior_fire_2_3_year = 1 WHERE rowid = 7274630
UPDATE weather_geo_no_fire SET  prior_fire_3_4_year = 1 WHERE rowid = 9123828
UPDATE weather_geo_no_fire SET  prior_fire_3_4_year = 1 WHERE rowid = 10955960
UPDATE weather_geo_no_fire SET  prior_fire_0_1_year = 1 WHERE rowid = 12775037
UPDATE weather_geo_no_fire SET  prior_fire_2_3_year = 1 WHERE rowid = 14601463
UPDATE weather_geo_no_fire SET  prior_fire_0_1_year = 1, prior_fire_3_4_year = 1 WHERE rowid = 16426606
UPDATE weather_geo_no_fire SET  prior_fire_1_2_year = 1 WHERE rowid = 18249826
UPDATE weather_geo_no_fire SET  prior_fire_2_3_year = 1 WHERE rowid = 20123128

### Export to CSV for Re-import and Google Vertex AI

In [27]:
df_no_fire = queries.get_df_no_fires()

df_no_fire.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
long,100000.0,-1.195949e+02,2.498189e+00,-1.243000e+02,-1.215000e+02,-1.200000e+02,-1.175000e+02,-1.142000e+02
lat,100000.0,3.724937e+01,2.579325e+00,3.260000e+01,3.510000e+01,3.700000e+01,3.950000e+01,4.200000e+01
month,100000.0,6.502060e+00,3.468013e+00,1.000000e+00,3.000000e+00,7.000000e+00,1.000000e+01,1.200000e+01
date,100000.0,1.247434e+09,1.733969e+08,9.466848e+08,1.097366e+09,1.248134e+09,1.398038e+09,1.546214e+09
precipitation,100000.0,1.381994e+00,5.063907e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.900000e-01,1.252400e+02
pressure,100000.0,9.206690e+01,6.522964e+00,7.222000e+01,8.799000e+01,9.280000e+01,9.735000e+01,1.026000e+02
humidity_2m,100000.0,5.305695e+00,2.086413e+00,6.100000e-01,3.780000e+00,5.130000e+00,6.650000e+00,1.740000e+01
temp_2m,100000.0,1.459133e+01,9.039807e+00,-1.721000e+01,8.230000e+00,1.403500e+01,2.114000e+01,4.262000e+01
temp_dew_point_2m,100000.0,2.260127e+00,6.254519e+00,-2.420000e+01,-1.950000e+00,2.730000e+00,6.910000e+00,2.256000e+01
temp_wet_bulb_2m,100000.0,8.425445e+00,6.494491e+00,-1.790000e+01,4.110000e+00,8.880000e+00,1.308000e+01,2.906000e+01


In [26]:
df_yes_fire = queries.get_df_yes_fires()

df_yes_fire.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
long,92659.0,-1.202729e+02,2.036115e+00,-1.243000e+02,-1.217000e+02,-1.204000e+02,-1.189000e+02,-1.144000e+02
lat,92659.0,3.834024e+01,2.639138e+00,3.260000e+01,3.660000e+01,3.870000e+01,4.070000e+01,4.200000e+01
month,92659.0,7.337873e+00,1.793029e+00,1.000000e+00,6.000000e+00,7.000000e+00,8.000000e+00,1.200000e+01
date,92659.0,1.236908e+09,1.703925e+08,9.466848e+08,1.088122e+09,1.222560e+09,1.386720e+09,1.545869e+09
precipitation,92659.0,5.650643e-01,2.048355e+00,0.000000e+00,0.000000e+00,1.000000e-02,3.300000e-01,1.090200e+02
pressure,92659.0,8.814598e+01,6.456601e+00,7.270000e+01,8.336000e+01,8.807000e+01,9.285000e+01,1.024400e+02
humidity_2m,92659.0,6.357436e+00,2.117470e+00,6.100000e-01,4.820000e+00,6.290000e+00,7.750000e+00,1.721000e+01
temp_2m,92659.0,2.018079e+01,6.122394e+00,-1.357000e+01,1.660000e+01,2.068000e+01,2.435000e+01,3.982000e+01
temp_dew_point_2m,92659.0,4.404742e+00,5.403795e+00,-2.420000e+01,9.100000e-01,4.880000e+00,8.290000e+00,2.203000e+01
temp_wet_bulb_2m,92659.0,1.229218e+01,4.762981e+00,-1.607000e+01,9.590000e+00,1.281000e+01,1.538000e+01,2.723000e+01


In [41]:
df_fires = pd.concat([df_no_fire, df_yes_fire], axis=0).sample(frac=1)
num_no_fire = len(df_fires[df_fires.fire_size_class == ''])
num_yes_fire = len(df_fires[df_fires.fire_size_class > ''])
print(f'Balanced classes of NO fires: {num_no_fire:,} and YES fire: {num_yes_fire:,}')

df_fires_export = df_fires.set_index(['date', 'long', 'lat'])
df_fires_export.to_csv('../data/df_fire_sample.csv')

Balanced classes of NO fires: 100,000 and YES fire: 92,659


### Acknowledgements

1. Short, Karen C. 2021. Spatial wildfire occurrence data for the United States, 1992-2018 [FPA_FOD_20210617]. 5th Edition. Fort Collins, CO: Forest Service Research Data Archive. https://doi.org/10.2737/RDS-2013-0009.5
Acknowledgements
2. These data were obtained from the NASA Langley Research Center (LaRC) POWER Project funded through the NASA Earth Science/Applied Science Program.
3. The U.S. Drought Monitor is produced through a partnership between the National Drought Mitigation Center at the University of Nebraska-Lincoln, the United States Department of Agriculture, and the National Oceanic and Atmospheric Administration.
4. This dataset utilizes the Harmonized World Soil Database by Fischer, G., F. Nachtergaele, S. Prieler, H.T. van Velthuizen, L. Verelst, D. Wiberg, 2008. Global Agro-ecological Zones Assessment for Agriculture (GAEZ 2008). IIASA, Laxenburg, Austria and FAO, Rome, Italy.
5. State of California Employeement Development Department for California County geospatial shape
6. California fire perimeters https://gis.data.ca.gov/datasets/CALFIRE-Forestry::california-fire-perimeters-all/about